In [1]:
print("hello")

hello


In [2]:
%pwd

'd:\\study\\GEN AI\\Medical Chatbot\\MedicalBot\\research'

In [ ]:
import os
os.chdir("../")


In [8]:
%pwd

'd:\\study\\GEN AI\\Medical Chatbot\\MedicalBot'

In [9]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


c:\Users\saiku\anaconda1\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
extracted_data = load_pdf_files("data")

In [12]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []

    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )

    return minimal_docs


In [13]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [16]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )

    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk


In [18]:
text_chunk= text_split(minimal_docs)
print(text_chunk[0])

page_content='The GALE
ENCYCLOPEDIA
of MEDICINE
SECOND EDITION' metadata={'source': 'data\\Medical_book.pdf'}


In [19]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )

    return embeddings

embedding = download_embeddings()


C:\Users\saiku\AppData\Local\Temp\ipykernel_19944\1601629471.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [21]:
print(embedding.embed_query("hello world"))

[-0.03447723388671875, 0.031023165211081505, 0.006734973751008511, 0.026108991354703903, -0.039362046867609024, -0.16030244529247284, 0.06692394614219666, -0.0064415112137794495, -0.04745052382349968, 0.014758817851543427, 0.07087530195713043, 0.05552754923701286, 0.019193369895219803, -0.026251282542943954, -0.01010951679199934, -0.02694047801196575, 0.022307446226477623, -0.022226616740226746, -0.1496925950050354, -0.017493078485131264, 0.007676276378333569, 0.054352231323719025, 0.0032544422429054976, 0.03172587975859642, -0.0846213772892952, -0.029405953362584114, 0.05159562826156616, 0.04812408611178398, -0.0033148485235869884, -0.05827920883893967, 0.04196930304169655, 0.022210706025362015, 0.128188818693161, -0.02233896777033806, -0.011656241491436958, 0.06292831897735596, -0.032876331359148026, -0.09122606366872787, -0.031175393611192703, 0.05269952118396759, 0.047034814953804016, -0.0842030942440033, -0.030056195333600044, -0.020744822919368744, 0.009517833590507507, -0.003721

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [24]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [26]:
from pinecone import Pinecone
Pinecone_api_key=PINECONE_API_KEY
pc=Pinecone(api_key=Pinecone_api_key)

In [27]:
pc

In [28]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [30]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunk,
    embedding=embedding,
    index_name=index_name
)

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [32]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='36c5c1c1-50b3-44e3-92fe-f844a0b1f13d', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='758c10cd-4142-44de-9ddc-88a52936f192', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='67a18006-cb29-43b9-a204-0183e243cb10', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

In [33]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")

In [34]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [35]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder caused by the abnormal release of a specific hormone from the pituitary gland, leading to increased growth in bone and soft tissue and other body disturbances. Gigantism occurs when this abnormality appears before bone growth stops, resulting in unusual height. If the abnormality arises after bone growth stops, the condition is known as acromegaly.


In [38]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

The treatment of acne depends on its severity. For mild noninflammatory acne, topical medications like tretinoin, benzoyl peroxide, adapalene, or salicylic acid are used to reduce comedone formation. For inflammatory acne, topical antibiotics may be added, and improvement is generally seen in two to four weeks.


In [39]:
response = rag_chain.invoke({"input": "who is saikumar?"})
print(response["answer"])

I don't know who Saikumar is based on the provided context.
